### Least Recently Used algorithm; commonly used as cache replacement policy

Would need the size of the cache. A data structure to store the cache and operations to rectrieve that data.First notion would be to somehow use a hashmap. Since it allows O(1) access times for insertions and retrieval. Needs more testing and code optimization and cleanup. Based upon the few tests run, looks good!

#### Notes:
##### Cache has a specific user defined size. Get and Set operations implemented

#### Approach:
###### Get:
1. Key is absent; return -1
2. Key is present; update LRU policy
        updateLRUPolicyonGet:
        if Required key is already at the top; do nothing
        else move the key to the top and shift all other keys down by 1 rank.

###### Set:
1. Key is already present; consider this as a get operation & update LRU policy as above
2. Key is absent, hence needs to be added; updateLRU policy then
        updateLRUPolicyonSet:
        if cache is not full; no need to remove any previous lru key. shift all keys down by 1 rank and add the new key to the top. 
        if cache is full; get the lru key; shift all keys down by 1; add the new key; remove the lru key
        
#### Data Structures Used:
##### HashMaps 

1. ranks_vs_keys: Used to maintain the main LRU policy to determine the order of recently accessed keys
2. keys_vs_ranks: Used to maintain a mapping of the ranks of the keys in terms of recently used
3. cache: main datastore for key:value pairs

In [1]:
import json

In [2]:
class LRUCache:

    def __init__(self, capacity):
        self.size = capacity
        self.used_capacity = 0
        self.ranks_vs_keys = {}
        self.keys_vs_ranks = {}
        self.cache = {}
        
    def get(self, key):
        if key not in self.cache.keys():
            val = -1
        else:
            self.updateLRUOnGet(key)
            val = self.cache[key]
        print(self)
        return val

    def put(self, key, value):
        if key in self.cache.keys():
            self.updateLRUOnGet(key)
        else:
            self.updateLRUOnSet(key, value)
        print(self)

    def updateLRUOnGet(self, key):
        if self.ranks_vs_keys[self.size] == key:
            return
        else:
            # update ranks of all keys above the rank of the key in the get request.
            curr_key_rank = self.keys_vs_ranks[key]
            new_key = key
            for rank in range(self.size, curr_key_rank - 1, -1):
                curr_key = self.ranks_vs_keys[rank]
                self.ranks_vs_keys[rank] = new_key
                self.keys_vs_ranks[new_key] = rank
                new_key = curr_key
    
    def updateLRUOnSet(self, key, val):
        if self.used_capacity < self.size:
            if self.used_capacity != 0:
                for rank in range(self.size - self.used_capacity, self.size):
                    next_key = self.ranks_vs_keys[rank + 1]
                    self.ranks_vs_keys[rank] = next_key
                    self.keys_vs_ranks[next_key] = rank
            self.ranks_vs_keys[self.size] = key
            self.keys_vs_ranks[key] = self.size
            self.used_capacity += 1
        else:
            least_recently_used_key = self.ranks_vs_keys[1]
            new_key = key
            for rank in range(self.size, 0, -1):
                curr_key = self.ranks_vs_keys[rank]
                self.ranks_vs_keys[rank] = new_key
                self.keys_vs_ranks[new_key] = rank
                new_key = curr_key
            del self.cache[least_recently_used_key]
            del self.keys_vs_ranks[least_recently_used_key]
        self.cache[key] = val
        
    def __repr__(self):
        return "CACHE: %s\nRANKS vs KEYS: %s\nKEYS vs RANKS: %s" % (json.dumps(self.cache),
                                                                  json.dumps(self.ranks_vs_keys),
                                                                  json.dumps(self.keys_vs_ranks))

In [3]:
l = LRUCache(4)

In [4]:
l.put(36, 72)

CACHE: {"36": 72}
RANKS vs KEYS: {"4": 36}
KEYS vs RANKS: {"36": 4}


In [5]:
l.put(2,11)

CACHE: {"2": 11, "36": 72}
RANKS vs KEYS: {"3": 36, "4": 2}
KEYS vs RANKS: {"2": 4, "36": 3}


In [6]:
l.put(10,20)

CACHE: {"2": 11, "36": 72, "10": 20}
RANKS vs KEYS: {"2": 36, "3": 2, "4": 10}
KEYS vs RANKS: {"2": 3, "36": 2, "10": 4}


In [7]:
l.put(7,14)

CACHE: {"2": 11, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 36, "2": 2, "3": 10, "4": 7}
KEYS vs RANKS: {"2": 2, "36": 1, "10": 3, "7": 4}


In [8]:
l.get(7)

CACHE: {"2": 11, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 36, "2": 2, "3": 10, "4": 7}
KEYS vs RANKS: {"2": 2, "36": 1, "10": 3, "7": 4}


14

In [9]:
l.get(36)

CACHE: {"2": 11, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 2, "2": 10, "3": 7, "4": 36}
KEYS vs RANKS: {"2": 1, "36": 4, "10": 2, "7": 3}


72

In [10]:
l.get(33)

CACHE: {"2": 11, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 2, "2": 10, "3": 7, "4": 36}
KEYS vs RANKS: {"2": 1, "36": 4, "10": 2, "7": 3}


-1

In [11]:
l.get(10)

CACHE: {"2": 11, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 2, "2": 7, "3": 36, "4": 10}
KEYS vs RANKS: {"2": 1, "36": 3, "10": 4, "7": 2}


20

In [12]:
l.put(5,17)

CACHE: {"5": 17, "36": 72, "10": 20, "7": 14}
RANKS vs KEYS: {"1": 7, "2": 36, "3": 10, "4": 5}
KEYS vs RANKS: {"5": 4, "36": 2, "10": 3, "7": 1}
